In [9]:
import os
import cv2
import face_recognition
import numpy as np
import torch
from facenet_pytorch import MTCNN
import mtcnn
import matplotlib.pyplot as plt
mtcnn = MTCNN(keep_all=True)
import json
import pyodbc as odbc
import shutil

In [10]:
SERVER_NAME = 'localhost\SQLEXPRESS'
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
DATABASE_NAME = 'Face_Recognition'
cnxn = odbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; \
                    SERVER='+SERVER_NAME+'; \
                    DATABASE = '+DATABASE_NAME+'; \
                    Trusted_connection=yes;')
cur = cnxn.cursor()

Create a database named 'Face_Detection' in your sql server

In [11]:
cur.execute("use Face_Detection")

In [12]:
def compare_faces(encoding1,encoding2):
    compare = face_recognition.compare_faces(encoding1,encoding2,tolerance=0.6)
    return compare

In [13]:
def main():
    matched_dir = input("Enter Matched_Images Folder Path: ")


    cur.execute("select * from Target_Images")
    Target_images = cur.fetchall()
    cur.execute("select * from All_Images")
    All_images = cur.fetchall()
    for target_image in Target_images:
        
        target_folder = os.path.join(matched_dir, target_image[2])
        if not os.path.exists(target_folder):
            os.makedirs(target_folder)

        cur.execute("select encodings from Target_images_encodings where person_id Like ?",(target_image[0]))
        pe = cur.fetchall()
        for pen in pe:
            person_encoding = np.array(json.loads(pen[0]))

            for image in All_images:
                cur.execute("select encodings from All_Images_encodings where image_id = ?",(image[0]))
                All_image_encodings = cur.fetchall()
                for gen in All_image_encodings:
                    All_image_encoding = np.array(json.loads(gen[0]))
                    match = compare_faces([All_image_encoding],person_encoding)
                    if (True in match) or ([True] in match):
                        cur.execute("SELECT 1 FROM matched_images WHERE Image_id = ? AND Person_ID = ? AND IMAGE_path = ? AND Name = ?", (image[0],target_image[0],image[1],target_image[2]))
                        if cur.fetchone() is None:
                            cur.execute("insert into matched_images(Image_path,Person_id,Image_id,Name) values (?,?,?,?)",(image[1],target_image[0],image[0],target_image[2]))
                            shutil.copy(image[1], target_folder)
                            break
    cur.commit()
    print("Successfull")
main()

Successfull
